## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [3]:
import os
from dotenv import load_dotenv
load_dotenv() ## loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="groq/compound",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000238EF7998A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000238EF7997B0>, model_name='groq/compound', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Utkarsh and I am a Chief AI Engineer")])

AIMessage(content='Hello Utkarsh! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 260, 'total_tokens': 312, 'completion_time': 0.10912, 'prompt_time': 0.008826, 'queue_time': 0.137338, 'total_time': 0.117946}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2827f9ad-a610-45f0-b824-4fe1154008fa-0', usage_metadata={'input_tokens': 260, 'output_tokens': 52, 'total_tokens': 312})

In [11]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Utkarsh and I am a Chief AI Engineer"),
        AIMessage(content='Hello Utkarsh! Nice to meet you. How can I assist you today?'),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="**Answer**\n\n- **Your name:** Utkarsh  \n- **Your role / what you do:** Chief AI Engineer  \n\n**Reasoning**\n\n1. In your very first message you introduced yourself:  \n   > “Hi , My name is Utkarsh and I am a Chief AI Engineer.”  \n\n2. From that statement we can directly extract two pieces of information:  \n   - The personal name is **Utkarsh**.  \n   - The professional title is **Chief AI Engineer**.\n\n3. When you later asked, “Hey, what's my name and what do I do?” the answer is simply the information we already have from step\u202f1.\n\nThus, based on the information you provided, your name is **Utkarsh** and you work as a **Chief AI Engineer**.", additional_kwargs={'reasoning_content': "\nTo answer the user's question, I need to recall the information provided earlier.\n\nThe user's name is Utkarsh, and he is a Chief AI Engineer.\n\nI don't need to use any tools to answer this question.\n\n\nYour name is Utkarsh, and you are a Chief AI Engineer."}, response

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [12]:
!pip install langchain_community

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [14]:
config={"configurable":{"session_id":"chat1"}}

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Utkarsh and I am a Chief AI Engineer")],
    config=config
)

In [16]:
response.content

'Hello Utkarsh! Nice to meet you. How can I assist you today?'

In [20]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Utkarsh.', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 364, 'total_tokens': 415, 'completion_time': 0.111412, 'prompt_time': 0.012076, 'queue_time': 0.109834, 'total_time': 0.123489}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7bee048f-bcd6-48db-8ae1-719babf2b49b-0', usage_metadata={'input_tokens': 364, 'output_tokens': 51, 'total_tokens': 415})

In [21]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I’m sorry, but I don’t have any information about your name. In our conversation so far you haven’t shared your name, and I have no way to infer it from the context. Therefore, I’m unable to answer “What’s my name?” with a specific name.'

In [27]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! How can I help you today?'

In [32]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.'

In [35]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config
)
response.content

'Your name is Utkarsh.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [5]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [8]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Utkarsh")]})

AIMessage(content='Hi Utkarsh! Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 286, 'total_tokens': 337, 'completion_time': 0.107231, 'prompt_time': 0.009108, 'queue_time': 0.101448, 'total_time': 0.116338}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f9aa0f40-9856-4914-9dfc-dd57a5376673-0', usage_metadata={'input_tokens': 286, 'output_tokens': 51, 'total_tokens': 337})

In [10]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [11]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Utkarsh")],
    config=config
)

response

AIMessage(content='Hi Utkarsh! Nice to meet you. How can I help you today?', additional_kwargs={'reasoning_content': '<Think>\n\n</Think>'}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 286, 'total_tokens': 353, 'completion_time': 0.151288, 'prompt_time': 0.009322, 'queue_time': 0.102318, 'total_time': 0.16061}, 'model_name': 'groq/compound', 'system_fingerprint': None, 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--baca1c06-b194-4856-9ae6-1a50c22e8c5b-0', usage_metadata={'input_tokens': 286, 'output_tokens': 67, 'total_tokens': 353})

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Utkarsh.'

In [13]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [17]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Chooot")],"language":"Hindi"})
response.content

'नमस्ते\u202fचूट! आपका परिचय सुनकर बहुत ख़ुशी हुई। मैं यहाँ आपके सवालों के जवाब देने, जानकारी साझा करने या किसी भी विषय पर बातचीत करने के लिए तैयार हूँ। आप मुझसे जो भी पूछना चाहें—चाहे वह सामान्य ज्ञान हो, तकनीकी मदद, भाषा सीखना हो, या बस कोई दिलचस्प चर्चा—बिल्कुल बेझिझक बताइए। मैं आपकी मदद करने के लिए यहाँ हूँ!'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [19]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [20]:
config = {"configurable": {"session_id": "chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Utkarsh")],"language":"Hindi"},
    config=config
)
response.content

'नमस्ते उत्कर्ष! आपसे मिलकर खुशी हुई। आप कैसे हैं?'

In [21]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [22]:
response.content

'**नाम:**\u202fउत्कर्ष  \n\n**तर्क:**  \n- आपने अपनी पहली संदेश में लिखा था, “Hi, I am Utkarsh”।  \n- इस वाक्य से स्पष्ट है कि आपका नाम **उत्कर्ष** है।  \n\nइसलिए आपका नाम **उत्कर्ष** ही है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"